In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('drive/My Drive')

In [3]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

### Data preparation

#### Load data

In [4]:
data = pd.read_excel('drive/My Drive/Colab Notebooks/My Drive/Online Retail.xlsx')

#### Explore data

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Explore the columns of the data
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
# Explore different regions of transactions
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

### Data Preprocessing

#### Data cleaning and transformation

In [ ]:
# Strip extra spaces in the description
data['Description'] = data['Description'].str.strip()

In [ ]:
# Drop the rows without any invoice number
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

In [ ]:
# Remove the cancelled transactions (those transactions containing 'C')
data = data[~data['InvoiceNo'].str.contains('C')]

<b>Note: </b> Tilde (~) means negation, i.e in this case, InvoiceNo that DON'T contains C

In [ ]:
# Define one-hot encoding function to make the data suitable for calculation. 
# For one-hot encoding, only there exist 0 and 1 in the fields. Make all positive values are 1 and others are 0
def onehot_encode(x):
    try:
        x = float(x)
    except ValueError:
        print('Not a number')
    if x <= 0:
        return 0
    else:
        return 1

### Mining Association Rules 

##### France

In [9]:
# Transactions done in France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [10]:
# Encoding the datasets
basket_France = basket_France.applymap(onehot_encode) 

In [11]:
# Mining Frequent itemsets
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
44,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000000,1.306667,0.017961,inf
258,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.051020,0.765306,0.051020,1.000000,1.306667,0.011974,inf
271,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...",(POSTAGE),0.053571,0.765306,0.053571,1.000000,1.306667,0.012573,inf
302,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
301,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
337,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.084184,0.127551,0.081633,0.969697,7.602424,0.070895,28.790816
335,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.084184,0.137755,0.081633,0.969697,7.039282,0.070036,28.454082
129,(RED RETROSPOT PICNIC BAG),(POSTAGE),0.071429,0.765306,0.068878,0.964286,1.260000,0.014213,6.571429
143,(SET OF 9 BLACK SKULL BALLOONS),(POSTAGE),0.066327,0.765306,0.063776,0.961538,1.256410,0.013015,6.102041
174,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122


<b> Comments: </b><br>
From the above output, it can be seen that paper cups and paper and plates are bought together in France. This is because the French have a culture of having a get-together with their friends and family atleast once a week. Also, since the French government has banned the use of plastic in the country, the people have to purchase the paper -based alternatives.

##### UK

In [15]:
# Transactions done in United Kingdom
basket_UK = (data[data['Country'] =="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [16]:
# Encoding the datasets
basket_UK = basket_UK.applymap(onehot_encode) 

In [17]:
# Mining Frequent itemsets
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
116,(BEADED CRYSTAL HEART PINK ON STICK),(DOTCOM POSTAGE),0.011036,0.037928,0.010768,0.975728,25.725872,0.010349,39.637371
2018,"(SUKI SHOULDER BAG, JAM MAKING SET PRINTED)",(DOTCOM POSTAGE),0.011625,0.037928,0.011196,0.963134,25.393807,0.010755,26.096206
2295,"(HERB MARKER THYME, HERB MARKER MINT)",(HERB MARKER ROSEMARY),0.010714,0.012375,0.010232,0.955000,77.173095,0.010099,21.947227
2302,"(HERB MARKER PARSLEY, HERB MARKER ROSEMARY)",(HERB MARKER THYME),0.011089,0.012321,0.010553,0.951691,77.240055,0.010417,20.444951
2300,"(HERB MARKER PARSLEY, HERB MARKER THYME)",(HERB MARKER ROSEMARY),0.011089,0.012375,0.010553,0.951691,76.905682,0.010416,20.443842
2276,"(HERB MARKER BASIL, HERB MARKER THYME)",(HERB MARKER ROSEMARY),0.010875,0.012375,0.010339,0.950739,76.828759,0.010205,20.048792
3357,"(REGENCY TEA PLATE PINK, REGENCY TEA PLATE ROSES)",(REGENCY TEA PLATE GREEN),0.012643,0.018000,0.011946,0.944915,52.496229,0.011719,17.827083
3375,"(WOODEN TREE CHRISTMAS SCANDINAVIAN, WOODEN HE...",(WOODEN STAR CHRISTMAS SCANDINAVIAN),0.012428,0.025660,0.011732,0.943966,36.787065,0.011413,17.388217
2290,"(HERB MARKER THYME, HERB MARKER MINT)",(HERB MARKER PARSLEY),0.010714,0.012214,0.010071,0.940000,76.960439,0.009940,16.463099
2278,"(HERB MARKER BASIL, HERB MARKER ROSEMARY)",(HERB MARKER THYME),0.011036,0.012321,0.010339,0.936893,76.039067,0.010203,15.650910


<b> Comments: </b><br>
Thyme, Rosemary, Mint,... are used for tea. That means, British people buy many kind of herbal tea. Furthermore, they often buy different colored tea-plates together. A reason behind this may be because typically the British enjoy tea very much and often collect different colored tea-plates for different occasions.

##### Sweden

In [19]:
# Transactions done in Sweden
basket_Sweden = (data[data['Country'] =="Sweden"]
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [20]:
# Encoding the datasets
basket_Sweden = basket_Sweden.applymap(onehot_encode) 

In [21]:
# Mining Frequent itemsets
frq_items = apriori(basket_Sweden, min_support = 0.05, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12 PENCILS SMALL TUBE SKULL),(PACK OF 72 SKULL CAKE CASES),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
1,(PACK OF 72 SKULL CAKE CASES),(12 PENCILS SMALL TUBE SKULL),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
4,(36 DOILIES DOLLY GIRL),(ASSORTED BOTTLE TOP MAGNETS),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
5,(ASSORTED BOTTLE TOP MAGNETS),(36 DOILIES DOLLY GIRL),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
180,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY CIRCUS PARADE),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
181,(CHILDRENS CUTLERY CIRCUS PARADE),(CHILDRENS CUTLERY DOLLY GIRL),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
212,(MOBILE VINTAGE HEARTS),(FAIRY CAKE FLANNEL ASSORTED COLOUR),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
213,(FAIRY CAKE FLANNEL ASSORTED COLOUR),(MOBILE VINTAGE HEARTS),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
216,(FIRST AID TIN),(PHARMACIE FIRST AID TIN),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf
217,(PHARMACIE FIRST AID TIN),(FIRST AID TIN),0.055556,0.055556,0.055556,1.0,18.0,0.052469,inf


<b> Comments: </b><br>
On analyzing the above rules, it is found that boys’ and girls’ cutlery are paired together. This makes practical sense because when a parent goes shopping for cutlery for his/her children, he/she would want the product to be a little customized according to the kid’s wishes.

#####  Portugal

In [22]:
# Transactions done in Portugal
basket_Portugal = (data[data['Country'] =="Portugal"]
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [23]:
# Encoding the datasets
basket_Portugal = basket_Portugal.applymap(onehot_encode) 

In [24]:
# Mining Frequent itemsets
frq_items = apriori(basket_Portugal, min_support = 0.05, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1170,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET 12 COLOUR PENCILS SPACEBOY),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1171,(SET 12 COLOUR PENCILS SPACEBOY),(SET 12 COLOUR PENCILS DOLLY GIRL),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1172,(SET OF 4 KNICK KNACK TINS LONDON),(SET 12 COLOUR PENCILS DOLLY GIRL),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1173,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET OF 4 KNICK KNACK TINS LONDON),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1174,(SET 12 COLOUR PENCILS DOLLY GIRL),(SET OF 4 KNICK KNACK TINS POPPIES),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1175,(SET OF 4 KNICK KNACK TINS POPPIES),(SET 12 COLOUR PENCILS DOLLY GIRL),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1176,(SET OF 4 KNICK KNACK TINS LONDON),(SET 12 COLOUR PENCILS SPACEBOY),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1177,(SET 12 COLOUR PENCILS SPACEBOY),(SET OF 4 KNICK KNACK TINS LONDON),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1178,(SET 12 COLOUR PENCILS SPACEBOY),(SET OF 4 KNICK KNACK TINS POPPIES),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf
1179,(SET OF 4 KNICK KNACK TINS POPPIES),(SET 12 COLOUR PENCILS SPACEBOY),0.051724,0.051724,0.051724,1.0,19.333333,0.049049,inf


<b> Comments: </b><br>
On analyzing the association rules for Portuguese transactions, it is observed that Tiffin sets (Knick Knack Tins) and color pencils. These two products typically belong to a primary school going kid. These two products are required by children in school to carry their lunch and for creative work respectively and hence are logically make sense to be paired together.

##### Canada

In [25]:
# Transactions done in Canada
basket_Canada = (data[data['Country'] =="Canada"]
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [26]:
# Encoding the datasets
basket_Canada = basket_Canada.applymap(onehot_encode) 

In [27]:
# Mining Frequent itemsets
frq_items = apriori(basket_Canada, min_support = 0.05, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

KeyboardInterrupt: ignored

<b> Comments: </b><br>
From the above output, it can be seen that paper cups and paper and plates are bought together in France. This is because the French have a culture of having a get-together with their friends and family atleast once a week. Also, since the French government has banned the use of plastic in the country, the people have to purchase the paper -based alternatives.


##### Australia

In [28]:
# Transactions done in Australia
basket_Australia = (data[data['Country'] =="Australia"]
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [29]:
# Encoding the datasets
basket_Australia = basket_Australia.applymap(onehot_encode) 

In [30]:
# Mining Frequent itemsets
frq_items = apriori(basket_Australia, min_support = 0.05, use_colnames = True)
  
# Mining strong association rules
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(10)

KeyboardInterrupt: ignored

### Take-home assignment

Carry out similar analyses for the remaining countries